### Trade Demo

#### Goal: 
- Create a domain node for canada
- Load the trade data for the country canada
- Upload trade on the domain node
- Create a Data Scientist User
- Accept/Decline any requests from the DS

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

schema = pd.read_csv("../datasets/schema.csv")
canada = pd.read_csv("../datasets/ca - feb 2021.csv")

/home/user/.pyenv/versions/3.8.1/envs/syft/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Step 1: Load the dataset

We have trade data for the country, which has provided data from Feb 2021. They key colums are:

- Commodity Code: the official code of that type of good
- Reporter: the country claiming the import/export value
- Partner: the country being claimed about
- Trade Flow: the direction of the goods being reported about (imports, exports, etc)
- Trade Value (US$): the declared USD value of the good

Let's have a quick look at the top five rows of the dataset.

In [2]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


### Step 2: Spin up the Domain Nodes

As the main requirement of this demo is that none of these countries should be sharing their data with anyone else, each country will get their own domain node to hold/protect their data while it's under study. So, we need to spin up 4 domain nodes.

Assuming you have [Docker](https://www.docker.com/) installed and configured with >=8GB of RAM, navigate to PySyft/packages/hagrid and run the following commands in separate terminals (can be done at the same time):


```bash
# install hagrid cli tool
pip install -r requirements.txt
pip install -e .
```

```bash
hagrid launch Canada --port=8081
```

<div class="alert alert-block alert-info">
    <b>Quick Tip:</b> Don't run this now, but later when you want to stop these nodes, you can simply run the same argument with the "stop" command. So from the PySyft/grid directory you would run. Note that these commands will delete the database by default. Add the flag "--keep_db=True" to keep the database around. Also note that simply killing the thread created by ./start is often insufficient to actually stop all nodes. Run the ./stop script instead. To stop the nodes listed above (and delete their databases) run:

```bash
hagrid land Canada
```
</div>

In [3]:
import syft as sy

# Let's login into the domain node
domain_client = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


In [4]:
required_columns = ["Classification", "Trade Flow", "Partner", "Commodity", "Commodity Code", "Trade Value (US$)"]
canada[:10000]

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,Panama,NaN,0804,"Dates, figs, pineapples, avocados, guavas, man...",0,NaN,NaN,576.0,469,0
9996,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,Peru,NaN,0804,"Dates, figs, pineapples, avocados, guavas, man...",0,NaN,NaN,3285911.0,3749290,0
9997,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,Philippines,NaN,0804,"Dates, figs, pineapples, avocados, guavas, man...",0,NaN,NaN,4480.0,48676,0
9998,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,Timor-Leste,NaN,0804,"Dates, figs, pineapples, avocados, guavas, man...",0,NaN,NaN,60.0,280,0


In [5]:
# Great !! we're logged in. Let's upload the dataset
domain_client.load_dataset(
    assets={"feb2020": canada[:10000]},
    name="Canada Trade Data - First 10000 rows",
    description="""A collection of reports from Canada's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

In [10]:
domain_client.datasets.all()

Unable to json decode message. Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
sy.lib.python.Dict

In [20]:
metadatas=[{'id': '1192bcd5-e7dc-4312-9fff-8f3b5c06c3ae', 'name': 'Canada Trade Data', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': 'a523e69c-c36e-427d-9b4f-2a5dcedd91d6', 'dtype': 'DataFrame', 'shape': '(1, 22)'}]}, {'id': '269043c0-9067-4557-9c6a-bfd834f1be2d', 'name': 'Canada Trade Data', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': 'f4956227-4869-4682-9c38-94f6cc664443', 'dtype': 'DataFrame', 'shape': '(1, 22)'}]}, {'id': '3789e774-b4d5-4902-ad4d-7096acd16f0e', 'name': 'Canada Trade Data', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': '440b9751-6b5f-48f0-869b-965dac2d8d9e', 'dtype': 'DataFrame', 'shape': '(1, 22)'}]}, {'id': 'cc28df5f-1b4c-4725-9762-2fb60c0740db', 'name': 'Canada Trade Data', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': 'e167e117-0926-4f93-b861-df012462f74b', 'dtype': 'DataFrame', 'shape': '(100, 22)'}]}, {'id': '6a49b4ed-0028-45d1-ad89-52b34350d257', 'name': 'Canada Trade Data', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': '4c89473a-f7bf-45db-bf51-63752517355f', 'dtype': 'DataFrame', 'shape': '(100, 6)'}]}, {'id': '524c9e2b-ad10-4e39-a772-dc3e997c526e', 'name': 'Canada Trade Data - First 5000 rows', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': '70313e75-0e29-437e-a735-0f279945d675', 'dtype': 'DataFrame', 'shape': '(5000, 22)'}]}, {'id': 'ff32d492-b3de-4e47-9af9-5377faef0bea', 'name': 'Canada Trade Data - First 5000 rows', 'manifest': '', 'description': "A collection of reports from Canada's statistics \n                    bureau about how much it thinks it imports and exports from other countries.", 'tags': [], 'str_metadata': {}, 'blob_metadata': {}, 'data': [{'name': 'feb2020', 'id': 'f5b07736-5cc2-4e5d-b130-326d39edae46', 'dtype': 'DataFrame', 'shape': '(5000, 22)'}]}]
for metadata in metadatas:
    metadata = sy.lib.python.Dict(metadata)
    for k, v in metadata.items():
        sy.serialize(v)
        